REMEMBER: FIRST CREATE A COPY OF THIS FILE WITH A UNIQUE NAME AND DO YOUR WORK THERE. AND MAKE SURE YOU COMMIT YOUR CHANGES TO THE `hw3_submissions` BRANCH.

# Assignment 3 | Cleaning and Exploring Data with Pandas



<img src="data/scoreCard.jpg" width=250>

In this assignment, you will investigate restaurant food safety scores for restaurants in San Francisco. Above is a sample score card for a restaurant. The scores and violation information have been made available by the San Francisco Department of Public Health. 

## Loading Food Safety Data


There are 2 files in the data directory:
1. business.csv containing food establishments in San Francisco
1. inspections.csv containing retaurant inspections records

Let's start by loading them into Pandas dataframes.  One of the files, business.csv, has encoding (ISO-8859-1), so you will need to account for that when reading it.

### Question 1

#### Question 1a
Read the two files noted above into two pandas dataframes named `bus` and `ins`, respectively. Print the first 5 rows of each to inspect them.


In [106]:
import pandas as pd

In [107]:
bus = pd.read_csv('data/businesses.csv', encoding='ISO-8859-1')
ins = pd.read_csv('data/inspections.csv')

In [108]:
bus.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494
3,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN
4,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051


In [109]:
ins.head()

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine


## Examining the Business data

From its name alone, we expect the `businesses.csv` file to contain information about the restaurants. Let's investigate this dataset.

### Question 2

#### Question 2a: How many records are there?

In [113]:
len(bus)

6315

#### Question 2b: How many unique business IDs are there?  

In [114]:
len(bus['business_id'].unique())

6315

#### Question 2c: What are the 5 most common businesses by name, and how many are there in San Francisco?

In [11]:
bus['name'].value_counts().head(5)

name
STARBUCKS COFFEE              72
PEET'S COFFEE & TEA           24
MCDONALDS                     12
SAN FRANCISCO SOUP COMPANY    11
WALGREENS                     11
Name: count, dtype: int64

## Zip code

Next, let's  explore some of the variables in the business table. We begin by examining the postal code.

### Question 3

#### Question 3a
How are the zip code values stored in python (i.e. data type)?

To answer this you might want to examine a particular entry.

In [115]:
type(bus['postal_code'][0])

str

#### Question 3b

What are the unique values of postal_code?

In [116]:
bus['postal_code'].unique()

array(['94104', '94109', '94133', '94110', '94122', '94103', '94115',
       '94131', '94111', '94117', '94107', '94108', '94102', '94132',
       '94105', '94134', '94124', '94116', '94121', '94118', '94112',
       '94123', '94114', '94127', '941102019', '941', '94545', '94066',
       '941033148', nan, '94158', '95105', '94013', '94130', 'CA',
       '92672', '94120', '94143', '94609', '94101', '00000', '94188',
       '94621', '94014', '94129', '94602', 'Ca'], dtype=object)

#### Question 3c

Let's say we decide to exclude the businesses that have no zipcode for our analysis (which might include food trucks for example).  Use the list of valid 5-digit zip codes below to create a new dataframe called bus_valid, with only businesses whose postal_codes show up in this list of valid zipcodes. How many businesses are there in this new dataframe?

In [117]:
validZip = ["94102", "94103", "94104", "94105", "94107", "94108",
            "94109", "94110", "94111", "94112", "94114", "94115",
            "94116", "94117", "94118", "94121", "94122", "94123", 
            "94124", "94127", "94131", "94132", "94133", "94134"]

In [118]:
bus_valid = bus[bus['postal_code'].isin(validZip)]

In [119]:
len(bus_valid)

5999

## Latitude and Longitude

Another aspect of the data we want to consider is the prevalence of missing values. If many records have missing values then we might be concerned about whether the nonmissing values are representative of the population.

### Question 4
 
Consider the longitude and latitude in the business DataFrame. 

#### Question 4a

How many businesses are missing longitude values, working with only the businesses that are in the list of valid zipcodes?

In [121]:
len(bus_valid[pd.isnull(bus_valid['longitude'])])

2483

#### Question 4b

Create a new dataframe with one row for each valid zipcode. The dataframe should include the following three columns:

1. `postal_code`: Contains the zip codes in the `validZip` variable above.
2. `null_lon`: The number of businesses in that zipcode with missing `longitude` values.
3. `not_null_lon`: The number of businesses without missing `longitude` values.

In [126]:
postal_codes = []
null_lon_counts = []
not_null_lon_counts = []

for entry in validZip:
    filtered_zips = bus_valid[bus_valid['postal_code'] == entry]
    
    null_lon_count = filtered_zips['longitude'].isnull().sum()
    not_null_lon_count = filtered_zips['longitude'].notnull().sum()
    
    postal_codes.append(entry)
    null_lon_counts.append(null_lon_count)
    not_null_lon_counts.append(not_null_lon_count)

byzip = pd.DataFrame({
    'postal_code': postal_codes,
    'null_lon': null_lon_counts,
    'not_null_lon': not_null_lon_counts
})
print(byzip)

   postal_code  null_lon  not_null_lon
0        94102       196           262
1        94103       252           306
2        94104        59            74
3        94105       100           126
4        94107       247           204
5        94108        82           142
6        94109       146           237
7        94110       251           334
8        94111       115           167
9        94112        59           131
10       94114       108           122
11       94115        76           162
12       94116        35            58
13       94117        67           130
14       94118       100           143
15       94121        55           106
16       94122       123           147
17       94123        53           116
18       94124        88            76
19       94127        28            43
20       94131        14            34
21       94132        65            69
22       94133       135           282
23       94134        29            45


#### 4c. Do any zip codes appear to have more than their 'fair share' of missing longitude? 

To answer this, you will want to compute the proportion of missing longitude values for each zip code, and print the proportion missing longitude, and print the top five zipcodes in descending order of proportion missing postal_code.


In [127]:
byzip['null_lon'] = byzip['null_lon'].astype(int)
byzip['not_null_lon'] = byzip['not_null_lon'].astype(int)

In [128]:
byzip['prop'] = byzip['null_lon']/byzip['not_null_lon']
byzip.sort_values(by = 'prop', ascending=False).head(5)

,postal_code,null_lon,not_null_lon,prop
4,94107,247,204,1.210784
18,94124,88,76,1.157895
21,94132,65,69,0.942029
10,94114,108,122,0.885246
16,94122,123,147,0.836735


# Investigate the inspection data

Let's now turn to the inspection DataFrame. Earlier, we found that `ins` has 4 columns, these are named `business_id`, `score`, `date` and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections. 

### Question 5

#### Question 5a
As with the business data, assess whether there is one inspection record for each business, by counting how many rows are in the data and how many unique businesses there are in the data. If they are exactly the same number, it means there is only one inspection per business, clearly.

In [129]:
len(ins) == len(ins['business_id'].unique())

False

#### Question 5b

What values does `type` take on? How many occurrences of each value is in the DataFrame? Create a new dataframe named `ins2` by copying `ins` and keeping only records with values of `type` that occur more than 10 times in the original table. In other words, eliminate records that have values of `type` that occur rarely (< 10 times). Check the result to make sure rare types are eliminated.

In [130]:
type(ins['type'][0])

str

In [131]:
ins['type'].value_counts()

type
routine      15429
complaint        1
Name: count, dtype: int64

In [132]:
ins2 = ins[ins['type'] == "routine"]
ins2['type'].value_counts() 

type
routine    15429
Name: count, dtype: int64

#### Question 5c

Since the data was stored in a .csv file, the dates are formatted as strings such as `20160503`. Once we read in the data, we would like to have dates in an appropriate format for analysis. Add a new column called `year` by capturing the first four characters of the date column. 

Hint: we have seen multiple ways of doing this in class, includings `str` operations, `lambda` functions, `datetime` operations, and others. Choose the method that works best for you :)

In [133]:
ins['year'] = ins['date'].astype(str).str[:4]

#### Question 5d

What range of years is covered in this data set? Are there roughly same number of inspections each year? Try dropping records for any years with less than 50 inspections and store the result in a new dataframe named `ins3`.

In [69]:
ins['year'].value_counts()
# includes years 2013-2016

year
2016    5840
2014    5629
2015    3923
2013      38
Name: count, dtype: int64

In [72]:
ins3 = ins[ins['year'] != "2013"]
ins3['year'].value_counts() # to confirm we removed 2013 entries

year
2016    5840
2014    5629
2015    3923
Name: count, dtype: int64

Let's examine only the inspections for one year: 2016. This puts businesses on a more equal footing because [inspection guidelines](https://www.sfdph.org/dph/eh/Food/Inspections.asp) generally refer to how many inspections should occur in a given year.

In [73]:
ins4 = ins[ins['year'] == "2016"]

### Question 6

#### Question 6a

Merge the business and 2016 inspections data, keeping all businesses regardless of whether they show up in the inspections file. Show the first several rows of the resulting dataframe.

In [75]:
merged2016 = pd.merge(bus, ins4, how="left", on=["business_id"])
merged2016.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number,score,date,type,year
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044,82.0,20160503.0,routine,2016
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262,94.0,20160513.0,routine,2016
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,98.0,20161005.0,routine,2016
3,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,96.0,20160311.0,routine,2016
4,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN,84.0,20160816.0,routine,2016


#### Question 6b
Print the 20 lowest rated businesses names, their addresses, and their ratings.

In [83]:
lowestrated = merged2016.sort_values(by = 'score', ascending=True).head(20)
lowestrated2 = lowestrated[['name', 'address', 'score']]
print(lowestrated2)

                             name              address  score
4892      GOLDEN RIVER RESTAURANT     5827 GEARY BLVD    52.0
7325                    POKI TIME     2101 LOMBARD ST    55.0
6281                 CRAZY PEPPER   2257 SAN JOSE AVE    55.0
4873                   UNCLE CAFE       65 WAVERLY PL    55.0
3008             HING WANG BAKERY        339 JUDAH ST    56.0
5627                   GOLDEN WOK   295 B ORIZABA AVE    56.0
5383                   TOP'S CAFE  2460 SAN BRUNO AVE    57.0
6067    L & G VIETNAMESE SANDWICH         602 EDDY ST    57.0
836                  RED A BAKERY      634 CLEMENT ST    57.0
4209      SAN TUNG RESTAURANT LLC      1031 IRVING ST    57.0
549           MA'S DIM SUM & CAFE      1315 POWELL ST    57.0
4903  NEW GARDEN RESTAURANT, INC.       716 KEARNY ST    57.0
6645    KOBANI MEDITERANEAN GRILL       3242 SCOTT ST    58.0
624           JOANIE'S DINER INC.   1329 COLUMBUS AVE    59.0
6938         PAK NAM THAI CUISINE       655 LARKIN ST    59.0
3563    

## Done!

Now commit this notebook to your `hw3_submissions` branch, push it to your GitHub repo, and open a PR!